In [111]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [112]:
df = pd.read_csv('../data/raw/heart.csv')

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [114]:
df["Sex"] = df["Sex"].map({"M": 1, "F": 0})
df["ExerciseAngina"] = df["ExerciseAngina"].map({"Y": 1, "N": 0})
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to the columns
df['ChestPainType'] = label_encoder.fit_transform(df['ChestPainType'])
df['RestingECG'] = label_encoder.fit_transform(df['RestingECG'])
df['ST_Slope'] = label_encoder.fit_transform(df['ST_Slope'])
# df = df.select_dtypes(include=["number"])


In [115]:
df.nunique(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    int64  
 2   ChestPainType   918 non-null    int64  
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    int64  
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    int64  
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    int64  
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(11)
memory usage: 86.2 KB


(Age                50
 Sex                 2
 ChestPainType       4
 RestingBP          67
 Cholesterol       222
 FastingBS           2
 RestingECG          3
 MaxHR             119
 ExerciseAngina      2
 Oldpeak            53
 ST_Slope            3
 HeartDisease        2
 dtype: int64,
 None)

In [116]:
# 2. Przygotuj X i y
X = df.drop(columns=["HeartDisease"])
y = df["HeartDisease"]

In [117]:
# 3. Podział na train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [118]:
# 4. Model XGBoost dla multiclass
model = xgb.XGBClassifier(
    objective="multi:softmax",      # można zmienić na "multi:softprob" jeśli chcesz prawdopodobieństwa
    num_class=4,                    # 4 klasy: 1, 2, 3, 4
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=42
)

In [119]:
# 5. Trening
model.fit(X_train, y_train)

c:\Users\Aleksander Misztal\Desktop\project\perceptron-numpy-keras-torch-benchmark\env\lib\site-packages\xgboost\training.py:183: UserWarning: [21:01:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=4, ...)

In [120]:
# 6. Predykcja i ewaluacja
y_pred = model.predict(X_val)

print("\n📊 Classification Report:")
print(classification_report(y_val, y_pred))

print("\n🧩 Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


📊 Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.84      0.83        82
           1       0.87      0.84      0.86       102

    accuracy                           0.84       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.84      0.84      0.84       184


🧩 Confusion Matrix:
[[69 13]
 [16 86]]
